# Tesla Ticker| Histroical Stock Price Dataset| GRU| BiGRU

In [1]:
# Import Libararies
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
# implort plotly for interactive plots
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import iplot
#import plot_acf
from statsmodels.graphics.tsaplots import plot_acf
# import seasonal decompose
from statsmodels.tsa.seasonal import seasonal_decompose
# scaler
from sklearn.preprocessing import MinMaxScaler
# train_test_split
from sklearn.model_selection import train_test_split
# sequential using keras
from tensorflow.keras.models import Sequential
# Dense, Bidirectional,GRU Layers using keras
from tensorflow.keras.layers import Dense, Bidirectional, GRU
# Adam optimizer
from tensorflow.keras.optimizers import Adam
# Early stoppng or callbacks
from tensorflow.keras.callbacks import EarlyStopping
# import metrics
from sklearn.metrics import r2_score, mean_squared_error
# import warnings
import warnings 
warnings.filterwarnings('ignore')

2024-06-20 17:33:29.010197: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-20 17:33:29.010414: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-20 17:33:29.195465: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## BiGRU

In [27]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, GRU, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Normalize the data
scaler = MinMaxScaler()
df['Close'] = scaler.fit_transform(df[['Close']])

# Split the dataset into training and testing sets
train_data, test_data = train_test_split(df['Close'], test_size=0.2, shuffle=False, random_state=42)

# Prepare input sequences and target values
def create_sequences(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size])
    return np.array(X), np.array(y)

window_size = 30
X_train, y_train = create_sequences(train_data.values, window_size)
X_test, y_test = create_sequences(test_data.values, window_size)

# Reshape input data to include the feature dimension
X_train = X_train.reshape(-1, window_size, 1)
X_test = X_test.reshape(-1, window_size, 1)

# Define the deeper BiGRU model with early stopping
model = Sequential([
    Bidirectional(GRU(64, return_sequences=True, input_shape=(window_size, 1))),
    Bidirectional(GRU(64)),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(optimizer=Adam(), loss='mse', metrics=['mae'])

# Define early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with early stopping
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Predictions
y_pred = model.predict(X_test)

# Evaluation
loss, mae = model.evaluate(X_test, y_test)
print('Test Loss:', loss)
print('Test MAE:', mae)

# Calculate R2 score
r2 = r2_score(y_test, y_pred)
print('R2 Score:', r2)

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 97ms/step - loss: 0.1124 - mae: 0.2437 - val_loss: 0.0053 - val_mae: 0.0648
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 0.0051 - mae: 0.0563 - val_loss: 0.0048 - val_mae: 0.0593
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 0.0031 - mae: 0.0406 - val_loss: 0.0025 - val_mae: 0.0432
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 0.0020 - mae: 0.0333 - val_loss: 0.0019 - val_mae: 0.0361
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 0.0015 - mae: 0.0285 - val_loss: 0.0014 - val_mae: 0.0297
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 0.0013 - mae: 0.0255 - val_loss: 0.0011 - val_mae: 0.0269
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 0.0011 - mae: 0.0242 - val_loss: 0.0011 - val_mae: 0.0271
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 0.0011 - mae: 0.0244 - val_loss: 0.0012 - val_mae: 0.0285
Epoch 9/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 0.00